In [2]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
# import mechanize  # Wrong use case
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options

In [3]:
import time

In [4]:
# Given a URL string, opens the URL in a headless Firefox instance and returns a reference to the webdriver object

def obtain_driver(url):
    buttons = []
    driver_options = Options()
    driver_options.headless = True

    driver = webdriver.Firefox(options=driver_options, executable_path=GeckoDriverManager().install())
    # driver.install_addon("~/.mozilla/firefox/f4t6w0s5.default-release/extensions/uBlock0@raymondhill.net.xpi", temporary=True)

    driver.get(url)
    
    time.sleep(1)
    
    return driver

In [5]:
# Given a DOI, uses obtain_driver to open up the search site and search the DOI to open up the paper in the page. Returns the webdriver associated with it.

def search_paper(doi):
    driver = obtain_driver("https://www.researchgate.net/search")
    
    search_field = driver.find_elements_by_class_name("search-form__input")[0]
    search_field.send_keys(doi)
    
#     time.sleep(1)
    
    enter_field = driver.find_elements_by_class_name("search-form__left-button")[0]
    driver.execute_script("arguments[0].click()", enter_field)
    
#     time.sleep(1)
    
    return driver

In [6]:
# Given the webdriver, shows more authors. Returns the modified webdriver.

def show_authors(driver): 
    
    time.sleep(1)
    
    first_start = time.time()
    
    buttons = driver.find_elements_by_class_name("show-more-authors")
    
    # Need to address the condition of the button not existing
    
    time.sleep(1)
 
    driver.execute_script("arguments[0].click();", buttons[0])
    
    time.sleep(1)

    first_end = time.time()

    print("DONE - " + str(first_end-first_start))
    
    return driver
# driver.quit()


In [7]:
# Given the webdriver, parses its source for author URLs.

def soup_it(driver):
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))
    
    driver.quit()

In [8]:
# Testing on an example

source = search_paper("10.1021/acs.iecr.5b03509")
searched = show_authors(source)
soup_it(searched)


Checking for linux64 geckodriver:v0.24.0 in cache
Driver found in /home/oru/.wdm/geckodriver/v0.24.0/linux64/geckodriver
DONE - 2.2347524166107178
https://www.researchgate.net/profile/Juliana_Coelho5
https://www.researchgate.net/profile/Ana_Ribeiro25
https://www.researchgate.net/profile/Alexandre_Ferreira4
https://www.researchgate.net/scientific-contributions/2082644774_Sebastiao_MP_Lucena
https://www.researchgate.net/profile/Alirio_Rodrigues
https://www.researchgate.net/profile/Diana_Azevedo


In [9]:
# Obtains author URLs from a basic request. Can't interact with the webpage to show more authors if there are > 4 authors.

def obtain_author_researchgate(website_url):
    request = urllib.request.Request(
        url = website_url,
        data = None,
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'
        }
    )

    testurl = urllib.request.urlopen(request)
    soup = BeautifulSoup(testurl, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))